## **Imports** 

In [35]:
import numpy as np 
import Markovitz_reduction 
import results_annalysis

## **Markovitz energie** 

We will test with 3 assets to verify the reduction 

Defining the expected return vector and the covaraiance matrix (only an example )

In [26]:


# Expected returns
mu_A = 0.08
mu_B = 0.12
mu_C = 0.10
# Covariances
sigma_AA = 0.0004;sigma_AB = 0.0003;sigma_AC = 0.0002;sigma_BA = 0.0003;sigma_BB = 0.0005
sigma_BC = 0.0004;sigma_CA = 0.0002;sigma_CB = 0.0004;sigma_CC = 0.0006
# Expected return vector
expected_return_vector = np.array([mu_A, mu_B, mu_C])
Mu = expected_return_vector

# Covariance matrix
covariance_matrix = np.array([[sigma_AA, sigma_AB, sigma_AC],
                              [sigma_BA, sigma_BB, sigma_BC],
                              [sigma_CA, sigma_CB, sigma_CC]])
V = covariance_matrix



Defining the optimizing parameters

In [27]:
Lamda1 = 1
Lamda2 = 0.5
fraction = 6
step = 0.01
iteration = 1000
n_cond_init = 50
temperature_fluctuation= 0.01
def a(self, t):
    a0=0
    return 0
    if t<70:
        return 0.1*t
    else:
        return 7

In [39]:
import Markovitz_reduction
markovitz = Markovitz_reduction.Markovitz(fraction,V, Mu, Lamda1, Lamda2, step, iteration, n_cond_init,temperature_fluctuation,a)
Weights = markovitz.Ising_to_Potfolio()

AttributeError: 'Markovitz' object has no attribute 'states'